<h1> Biomass - Erko Rotoy Single Case Analysis </h1>

In [ ]:
import os
import sys
import random
import math
import pickle
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import keras
import skimage.io
from tqdm import tqdm

LIB_DIRECTORY = '/root/alok/repos/cv_research/lib/'
sys.path.insert(0, os.path.join(LIB_DIRECTORY, 'maskrcnn'))
from mrcnn.config import Config
import mrcnn.utils as utils
import mrcnn.model as modellib
import mrcnn.visualize as visualize
from mrcnn.model import log
import mcoco.coco as coco
# import mextra.utils as extra_utils
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import json
import os
import glob
import sys
LIB_DIRECTORY = '/root/alok/repos/cv_research/lib/'
sys.path.insert(0, LIB_DIRECTORY)

import pandas as pd
from pycocotools.coco import COCO

%matplotlib inline
%config IPCompleter.greedy=True
BASE_DIR = '/root/data/models/erko/mask_rcnn_instance_segmentation'
DATA_DIR = '/root/data/erko/'
WEIGHTS_DIR = os.path.join(BASE_DIR, "weights")
MODEL_DIR = os.path.join(BASE_DIR, "logs", "body_part_segmentation_20181031_21H02")



<h1> Load Data </h1>

In [ ]:
annotation_file = '/root/alok/data/images/annotation_file_test_set.json'
coco_output = json.load(open(annotation_file))

<h1> Set up data and helper functions </h1>

In [ ]:
'''
Get stereo frame pairs where for each one, we have the left image ID and right image ID
'''
coco = COCO(annotation_file)
image_ids = coco.getImgIds()
images = coco.loadImgs(image_ids)
stereo_frame_pairs = {}
for image in images:
    local_path = image['local_path']
    sfp_id = int(local_path.split('/')[-3])
    side = 'left' if 'left' in local_path else 'right'
    if sfp_id not in list(stereo_frame_pairs.keys()):
        stereo_frame_pairs[sfp_id] = {}
    stereo_frame_pairs[sfp_id][side] = image['id']
    
stereo_frame_pairs = list(stereo_frame_pairs.values())
    
    

In [ ]:
'''
Define some helper functions (most of this code is adapted from cv_algorithms)
'''


import numpy as np
import pandas as pd
import statsmodels.api as sm
from PIL import Image, ImageDraw
from scipy.optimize import linear_sum_assignment
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances


FOCAL_LENGTH = 0.0107
BASELINE = 0.135
PIXEL_SIZE_M = 3.45 * 1e-6
FOCAL_LENGTH_PIXEL = FOCAL_LENGTH / PIXEL_SIZE_M
IMAGE_SENSOR_WIDTH = 0.01412
IMAGE_SENSOR_HEIGHT = 0.01412
RANDOM_STATE = 170
DENSITY = 1e6  # g per cubic meter
COST_THRESHOLD = 100.0  # another magic number

CATEGORIES = [
    {'id': 1, 'name': 'salmon'},
    {'id': 2, 'name': 'Head'},
    {'id': 3, 'name': 'Caudal Fin'},
    {'id': 4, 'name': 'Dorsal Fin'},
    {'id': 5, 'name': 'Adipose Fin'},
    {'id': 6, 'name': 'Anal Fin'},
    {'id': 7, 'name': 'Pelvic Fin'},
    {'id': 8, 'name': 'Pectoral Fin'},
    {'id': 9, 'name': 'Eye'}
]

SALMON_CATEGORY_ID = [cat['id'] for cat in CATEGORIES if cat['name'] == 'salmon'][0]

def convert_to_world_point(x, y, d):
    """ from pixel coordinates to world coordinates """
    # TODO (@Thomas) this is hard coded and this bad....
    image_center_x = 3000 / 2.0  # depth_map.shape[1] / 2.0
    image_center_y = 4096 / 2.0  # depth_map.shape[0] / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (IMAGE_SENSOR_WIDTH / 3000)
    sensor_z = px_z * (IMAGE_SENSOR_HEIGHT / 4096)

    # d = depth_map[y, x]
    world_y = d
    world_x = (world_y * sensor_x) / FOCAL_LENGTH
    world_z = (world_y * sensor_z) / FOCAL_LENGTH
    return np.array([world_x, world_y, world_z])


def depth_from_disp(disp):
    depth = FOCAL_LENGTH_PIXEL*BASELINE / np.array(disp)
    return depth


def match_body_parts(annotations, categories, whole_fish="salmon"):
    
    """ take the predictions on a single frame and associate body parts with whole salmon using a unique identifier"""
    
    whole_fish_id = [cat for cat in categories if cat['name'] == whole_fish][0]['id']
    fish = []
    body_parts = []
    fish_counter = 0

    # first we separate whole salmon from body parts and give a unique identifier
    for annotation in annotations:
        if annotation['category_id'] == whole_fish_id:
            annotation['fish_id'] = fish_counter
            fish.append(annotation)
            fish_counter += 1
        else:
            body_parts.append(annotation)

    # second we match body parts with whole salmon
    # @TODO (Thomas) this is not great because it does not take edges cases into account.
    for part in body_parts:
        bbox = part['bbox']
        part_centroid = [np.mean([bbox[0], bbox[2]]), np.mean([bbox[1], bbox[3]])]
        for f in fish:
            bbox = f['bbox']
            if bbox[0] < part_centroid[0] < bbox[2] and bbox[1] < part_centroid[1] < bbox[3]:
                part['fish_id'] = f['fish_id']
                part['fish_ann_id'] = f['id']
                break

    matched_body_pars = [ann for ann in annotations if 'fish_id' in ann.keys()]
    return matched_body_pars


def left_right_matching(left_annotations, right_annotations, categories, whole_fish="salmon"):
    
    """match the bboxes. Return a list of matched bboxes"""
    
    # get the salmon category id
    whole_fish_id = [cat for cat in categories if cat['name'] == whole_fish][0]['id']

    # let's use x1, x2 to match bboxes
    left_centroids = []
    left_ids = []
    for ann in left_annotations:
        if ann['category_id'] != whole_fish_id:
            continue
        bbox = ann['bbox']
        # centroid = [(bbox[3] - bbox[1])/2.0, (bbox[2] - bbox[0])/2.0]
        corner = [bbox[2], bbox[0]]
        left_ids.append(ann['fish_id'])
        left_centroids.append(corner)
            

    right_centroids = []
    right_ids = []
    for ann in right_annotations:
        if ann['category_id'] != whole_fish_id:
            continue
        bbox = ann['bbox']
        # centroid = [(bbox[3] - bbox[1])/2.0, (bbox[2] - bbox[0])/2.0]
        corner = [bbox[2], bbox[0]]
        right_ids.append(ann['fish_id'])
        right_centroids.append(corner)

    # euclidean distance in (x1, x2) space
    cost_matrix = euclidean_distances(left_centroids, right_centroids)
    
    # hungarian algorithm to minimize weights in bipartite graph
    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    matched_annotations = []
    for (r, c) in zip(row_ind, col_ind):
        if cost_matrix[r, c] < COST_THRESHOLD:
            left_matched_parts = [l for l in left_annotations if l['fish_id'] == left_ids[r] and l['category_id'] != whole_fish_id]
            right_matched_parts = [r for r in right_annotations if r['fish_id'] == right_ids[c] and r['category_id'] != whole_fish_id]
            matched_annotations.append([left_matched_parts, right_matched_parts])

    return matched_annotations


def calculate_pairwise_distances(left_matched_parts, right_matched_parts, categories):
    
    """ Create a dataframe of pairwise distances for each fish detection in this image pair """
    
    left_category_ids = [ann['category_id'] for ann in left_matched_parts]
    right_category_ids = [ann['category_id'] for ann in right_matched_parts]

    # initialize pairwise distances for this fish detection
    category_ids = [str(cat['id']) for cat in categories if cat['name'] != 'salmon']
    distances = [c + k for (i, c) in enumerate(category_ids) for k in category_ids[i + 1:]]
    dataset = {}
    for d in distances:
        dataset[d] = np.nan

    # calculate disparities & world coordinates
    world_coordinates = {}
    for cat in categories:
        # calculate left centroid
        left_parts = [part for part in left_matched_parts if part['category_id'] == cat['id']]
        right_parts = [part for part in right_matched_parts if part['category_id'] == cat['id']]
        if len(left_parts) > 1 or len(right_parts) > 1:
            
            # more than one body part of this category has been found on this fish -- skip this body part
            
            continue
        
        if left_parts and right_parts:
            left_part, right_part = left_parts[0], right_parts[0]
            
            # calculate left centroid
            seg = left_part['segmentation'][0]
            poly = np.array(seg).reshape((int(len(seg) / 2), 2))
            p = [(r[0], r[1]) for r in poly]
            left_mask = Image.new('L', (4096, 3000), 0)
            ImageDraw.Draw(left_mask).polygon(p, outline=1, fill=1)
            left_mask = np.array(left_mask)
            x, y = np.nonzero(left_mask)
            left_centroid = [np.mean(x), np.mean(y)]

            # calculate right centroid
            seg = right_part['segmentation'][0]
            poly = np.array(seg).reshape((int(len(seg) / 2), 2))
            p = [(r[0], r[1]) for r in poly]
            right_mask = Image.new('L', (4096, 3000), 0)
            ImageDraw.Draw(right_mask).polygon(p, outline=1, fill=1)
            right_mask = np.array(right_mask)
            x, y = np.nonzero(right_mask)
            right_centroid = [np.mean(x), np.mean(y)]

            # calculate disparity
            disparities = np.abs(left_centroid[1] - right_centroid[1])
            depth = depth_from_disp(disparities)
            if left_part['fish_ann_id'] == 13999 and left_part['category_id'] == 2:
                print('Disparity: {}'.format(disparities))
                print('Depth: {}'.format(depth))
            world_coordinates[str(cat['id'])] = convert_to_world_point(left_centroid[0], left_centroid[1], depth)

    if left_matched_parts[0]['fish_ann_id'] == 13999:
        print(world_coordinates['2'], world_coordinates['4'])
    
    # now calculate the pairwise distances
    for pair in dataset.keys():
        cat0, cat1 = pair[0], pair[1]
        if cat0 in world_coordinates and cat1 in world_coordinates:
            dist = np.linalg.norm(world_coordinates[cat0] - world_coordinates[cat1])
            dataset[pair] = dist

    return dataset




<h1> Analyze one case (i.e. one stereo frame pair) </h1>

In [ ]:
stereo_frame_pair_idx = 1100


stereo_frame_pair = stereo_frame_pairs[stereo_frame_pair_idx]
left_image_id = stereo_frame_pair['left']
left_image = coco.loadImgs([left_image_id])
left_annotation_ids = coco.getAnnIds(imgIds=[left_image_id])
left_annotations = coco.loadAnns(left_annotation_ids)

right_image_id = stereo_frame_pair['right']
right_image = coco.loadImgs([left_image_id])
right_annotation_ids = coco.getAnnIds(imgIds=[right_image_id])
right_annotations = coco.loadAnns(right_annotation_ids)
categories = CATEGORIES

left_annotations = match_body_parts(left_annotations, categories)
right_annotations = match_body_parts(right_annotations, categories)
if left_annotations and right_annotations:
    matched_annotations = left_right_matching(left_annotations, right_annotations, categories)
    for pair in matched_annotations:
        if pair[0] and pair[1]:
            dataset = calculate_pairwise_distances(pair[0], pair[1], categories)




